# Exercise

In [ ]:
#1
#Implement PCA from scratch

In [ ]:
#2
#Extend #1 and implement incremental PCA
#which is basically feeding the PCA mini-batch at a time
#compare the time with #1 and efficiency

In [ ]:
#3
#Extend #2 to support KernelPCA for rbf kernel

In [ ]:
#4
#Implement ISOmap from scratch

In [ ]:
#5
#Implement Tsne from scratch

# Solution